# show all pages

```text
all pages pdf (type: png) in path: data/pages
```

In [ ]:
# %pip install PyMuPDF matplotlib

In [2]:
import fitz  # PyMuPDF
import os
import matplotlib.pyplot as plt
from PIL import Image

# === 1. Render tất cả các trang thành ảnh ===
pdf_path = "../data/pdfs/file_2.pdf"
output_folder = "../data/images"
os.makedirs(output_folder, exist_ok=True)

pdf_file = fitz.open(pdf_path)
zoom = 2  # 2 = 144 DPI, có thể tăng lên nếu cần nét hơn
mat = fitz.Matrix(zoom, zoom)

saved_images = []

for page_index in range(len(pdf_file)):
    page = pdf_file[page_index]
    pix = page.get_pixmap(matrix=mat)
    output_path = os.path.join(output_folder, f"page_{page_index + 1}.png")
    pix.save(output_path)
    saved_images.append(output_path)

pdf_file.close()

# === 2. Hiển thị TẤT CẢ ảnh bằng matplotlib ===
print(f"📂 Đã lưu {len(saved_images)} ảnh, bắt đầu hiển thị...")

# # Thiết lập lưới: 12 cột, bao nhiêu hàng tùy số ảnh
# cols = 6
# rows = (len(saved_images) + cols - 1) // cols

# fig, axes = plt.subplots(rows, cols, figsize=(cols * 2.5, rows * 2.5))
# axes = axes.flatten()

# for ax, img_path in zip(axes, saved_images):
#     img = Image.open(img_path)
#     ax.imshow(img)
#     ax.axis("off")
#     ax.set_title(os.path.basename(img_path).split("_")[1], fontsize=6, pad=2)

# # Ẩn các ô thừa
# for ax in axes[len(saved_images):]:
#     ax.axis("off")

# plt.tight_layout()
# plt.show()

📂 Đã lưu 168 ảnh, bắt đầu hiển thị...


In [ ]:
import cv2
import numpy as np
from pathlib import Path

# ==========================
# 🔹 Đường dẫn
# ==========================
input_folder = Path("../src/data/ground_struct/pages_images")
output_folder = Path("../src/data/ground_struct/pages_images_processed")
output_folder.mkdir(parents=True, exist_ok=True)

# ==========================
# 🔹 Hàm tiền xử lý đơn giản
# ==========================
def simple_threshold(image_path, output_path):
    """
    Chỉ grayscale + threshold đơn giản
    """
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Không đọc được: {image_path}")
    
    # 1. Grayscale
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    # 2. Otsu threshold (tự động tìm ngưỡng tối ưu)
    _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    # 3. Đảo màu nếu nền đen
    if np.mean(binary) < 127:
        binary = cv2.bitwise_not(binary)
    
    cv2.imwrite(str(output_path), binary)
    return binary

# ==========================
# 🔹 Xử lý batch
# ==========================
processed = 0
failed = []

for img_file in sorted(input_folder.glob("page_*.png")):
    output_path = output_folder / img_file.name
    
    try:
        simple_threshold(img_file, output_path)
        processed += 1
        print(f"✅ {img_file.name}")
    except Exception as e:
        failed.append(img_file.name)
        print(f"❌ {img_file.name}: {e}")

print(f"\n📊 Đã xử lý: {processed} → {output_folder}")
if failed:
    print(f"❌ Thất bại: {failed}")

In [ ]:
import cv2
import numpy as np
from pathlib import Path

input_folder = Path("../src/data/ground_struct/pages_images")
output_folder = Path("../src/data/ground_struct/pages_images_processed")
output_folder.mkdir(parents=True, exist_ok=True)

def threshold_methods(image_path, output_path, method="otsu"):
    """
    method: 'otsu', 'adaptive_mean', 'adaptive_gaussian', 'simple'
    """
    img = cv2.imread(str(image_path))
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    
    if method == "otsu":
        _, binary = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
    
    elif method == "adaptive_mean":
        binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_MEAN_C, 
                                      cv2.THRESH_BINARY, 11, 2)
    
    elif method == "adaptive_gaussian":
        binary = cv2.adaptiveThreshold(gray, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, 
                                      cv2.THRESH_BINARY, 11, 2)
    
    elif method == "simple":
        _, binary = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY)
    
    else:
        raise ValueError(f"Method không hợp lệ: {method}")
    
    # Đảo màu nếu cần
    if np.mean(binary) < 127:
        binary = cv2.bitwise_not(binary)
    
    cv2.imwrite(str(output_path), binary)
    return binary

# Xử lý batch
processed = 0
failed = []

for img_file in sorted(input_folder.glob("page_*.png")):
    output_path = output_folder / img_file.name
    
    try:
        # Thử Otsu trước
        threshold_methods(img_file, output_path, method="otsu")
        processed += 1
        print(f"✅ {img_file.name}")
    except Exception as e1:
        try:
            # Fallback: Adaptive Gaussian
            threshold_methods(img_file, output_path, method="adaptive_gaussian")
            processed += 1
            print(f"✅ {img_file.name} (adaptive)")
        except Exception as e2:
            failed.append(img_file.name)
            print(f"❌ {img_file.name}: {e2}")

print(f"\n📊 Đã xử lý: {processed} → {output_folder}")
if failed:
    print(f"❌ Thất bại: {failed}")

# all page + all images

In [ ]:
import fitz  # PyMuPDF
import os
from PIL import Image
import matplotlib.pyplot as plt

# === 1. Render toàn bộ trang thành ảnh ===
pdf_path = "../src/data/ground_struct/file_2.pdf"
output_pages = "../src/data/ground_struct/pages_images"
output_embedded = "../data/images"
os.makedirs(output_pages, exist_ok=True)
os.makedirs(output_embedded, exist_ok=True)

pdf_file = fitz.open(pdf_path)
zoom = 4
mat = fitz.Matrix(zoom, zoom)

saved_pages = []
saved_embedded = []

# --- Render trang ---
for page_index in range(len(pdf_file)):
    page = pdf_file[page_index]
    pix = page.get_pixmap(matrix=mat)
    page_path = os.path.join(output_pages, f"page_{page_index + 1}.png")
    pix.save(page_path)
    saved_pages.append(page_path)

# --- Trích xuất ảnh nhúng ---
for page_index in range(len(pdf_file)):
    page = pdf_file[page_index]
    image_list = page.get_images(full=True)
    if image_list:
        print(f"[+] Page {page_index + 1}: {len(image_list)} embedded image(s)")
        for img_index, img in enumerate(image_list, start=1):
            xref = img[0]
            base_image = pdf_file.extract_image(xref)
            image_bytes = base_image["image"]
            image_ext = base_image["ext"]
            image_name = f"page_{page_index + 1}_image_{img_index}.{image_ext}"
            image_path = os.path.join(output_embedded, image_name)
            with open(image_path, "wb") as f:
                f.write(image_bytes)
            saved_embedded.append(image_path)
    else:
        print(f"[!] Page {page_index + 1}: no embedded images")

pdf_file.close()

# === 2. Hiển thị tất cả ảnh nhúng bằng matplotlib ===
print(f"📂 Đã lưu {len(saved_embedded)} ảnh nhúng, bắt đầu hiển thị...")

cols = 6
rows = (len(saved_embedded) + cols - 1) // cols

fig, axes = plt.subplots(rows, cols, figsize=(cols * 2.5, rows * 2.5))
axes = axes.flatten()

for ax, img_path in zip(axes, saved_embedded):
    img = Image.open(img_path)
    ax.imshow(img)
    ax.axis("off")
    ax.set_title(os.path.basename(img_path), fontsize=6, pad=2)

# Ẩn ô thừa
for ax in axes[len(saved_embedded):]:
    ax.axis("off")

plt.tight_layout()
plt.show()

# all contents

In [ ]:
import fitz  # PyMuPDF
import os

pdf_path = "../src/data/ground_struct/file_2.pdf"
output_dir = "../src/data/contents"
os.makedirs(output_dir, exist_ok=True)

pdf_file = fitz.open(pdf_path)

blocks = [] #  Lưu trữ tất cả các block văn bản theo mỗi page

for page_number in range(1, len(pdf_file) + 1):
    page = pdf_file[page_number - 1]
    text = page.get_text("text")
    text_blocks = page.get_text_blocks()
    text_path = os.path.join(output_dir, f"page_{page_number}.txt")
    with open(text_path, "w", encoding="utf-8") as f:
        for block in text_blocks:
            f.write(block[4])  # Write the text content of each block
            blocks.append(np.array([block]))  # Lưu block vào danh sách
    print(f"✅ Saved page_{page_number}.txt")

pdf_file.close()

In [ ]:
import numpy as np

blocks[3]

In [ ]:
import cv2
import numpy as np
from pathlib import Path

# ==========================
# 🔹 Định nghĩa đường dẫn
# ==========================
input_folder = Path("../src/data/ground_struct/pages_images")
output_folder = Path("../src/data/ground_struct/pages_images_processed")
output_folder.mkdir(parents=True, exist_ok=True)

# ==========================
# 🔹 Advanced Grayscale
# ==========================
def advanced_grayscale(img):
    """
    Chuyển ảnh màu sang grayscale thông minh, giữ độ tương phản cao
    và chi tiết chữ từ vùng màu khác nhau.
    """
    lab = cv2.cvtColor(img, cv2.COLOR_BGR2LAB)
    L, A, B = cv2.split(lab)

    # Tăng cường độ sáng (CLAHE chỉ trên kênh L)
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    L_enhanced = clahe.apply(L)

    # Kết hợp lại và chuyển sang grayscale
    lab_enhanced = cv2.merge([L_enhanced, A, B])
    bgr_enhanced = cv2.cvtColor(lab_enhanced, cv2.COLOR_LAB2BGR)
    gray = cv2.cvtColor(bgr_enhanced, cv2.COLOR_BGR2GRAY)

    # Cải thiện tương phản cục bộ
    gray = cv2.addWeighted(gray, 1.5, cv2.GaussianBlur(gray, (0, 0), 3), -0.5, 0)
    return gray

# ==========================
# 🔹 Phương pháp chính
# ==========================
def preprocess_for_ocr(image_path, output_path):
    """
    Tiền xử lý ảnh tối ưu cho OCR tiếng Việt (tăng độ rõ nét, giảm nhiễu, giữ cấu trúc văn bản)
    """
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Không thể đọc ảnh: {image_path}")

    # 1. Grayscale nâng cao
    gray = advanced_grayscale(img)

    # 2. Resize nếu ảnh nhỏ
    height, width = gray.shape
    if height < 1200 or width < 1200:
        scale_factor = max(1200/height, 1200/width)
        gray = cv2.resize(gray, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)

    # 3. Khử nhiễu nhẹ nhàng nhưng giữ biên chữ
    denoised = cv2.bilateralFilter(gray, 9, 75, 75)

    # 4. Tăng cường độ tương phản bằng CLAHE
    clahe = cv2.createCLAHE(clipLimit=3.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(denoised)

    # 5. Làm sắc nét (kernel tự nhiên, không quá mạnh)
    kernel_sharpen = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    sharpened = cv2.filter2D(enhanced, -1, kernel_sharpen)

    # 6. Adaptive Threshold - xử lý ánh sáng không đều
    binary = cv2.adaptiveThreshold(sharpened, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 15, 2)

    # 7. Làm sạch nhiễu nhỏ, nối nét chữ
    kernel = np.ones((2, 2), np.uint8)
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)
    cleaned = cv2.morphologyEx(cleaned, cv2.MORPH_OPEN, kernel)

    # 8. Kiểm tra nền sáng/tối để đảo màu nếu cần
    if np.mean(cleaned) < 127:
        cleaned = cv2.bitwise_not(cleaned)

    cv2.imwrite(str(output_path), cleaned)
    return cleaned

# ==========================
# 🔹 Phương pháp thay thế
# ==========================
def preprocess_for_ocr_alternative(image_path, output_path):
    """
    Phiên bản đơn giản, vẫn hiệu quả cho tài liệu sáng màu hoặc có vùng màu đồng đều.
    """
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Không thể đọc ảnh: {image_path}")

    gray = advanced_grayscale(img)

    # Resize nếu cần
    height, width = gray.shape
    if height < 1000:
        scale_factor = 1000 / height
        gray = cv2.resize(gray, None, fx=scale_factor, fy=scale_factor, interpolation=cv2.INTER_CUBIC)

    # Cân bằng histogram toàn cục
    equalized = cv2.equalizeHist(gray)

    # Khử nhiễu nhanh
    denoised = cv2.fastNlMeansDenoising(equalized, None, h=8, templateWindowSize=7, searchWindowSize=21)

    # Làm sắc nét nhẹ
    kernel = np.array([[0, -1, 0],
                       [-1, 5, -1],
                       [0, -1, 0]])
    sharpened = cv2.filter2D(denoised, -1, kernel)

    cv2.imwrite(str(output_path), sharpened)
    return sharpened

# ==========================
# 🔹 Chạy batch
# ==========================
processed_count = 0
failed_files = []

for img_file in sorted(input_folder.glob("page_*.png")):
    output_path = output_folder / img_file.name

    try:
        preprocess_for_ocr(img_file, output_path)
        processed_count += 1
        print(f"✅ Đã xử lý: {img_file.name}")
    except Exception as e:
        try:
            preprocess_for_ocr_alternative(img_file, output_path)
            processed_count += 1
            print(f"✅ Đã xử lý (alt): {img_file.name}")
        except Exception as e2:
            failed_files.append(img_file.name)
            print(f"❌ Lỗi khi xử lý {img_file.name}: {e2}")

print(f"\n📊 Tổng kết: Đã xử lý {processed_count} ảnh → {output_folder}")
if failed_files:
    print(f"❌ Thất bại: {len(failed_files)} file: {failed_files}")


In [ ]:
import cv2
import numpy as np
from pathlib import Path

# ==============================================================
# 🔹 Advanced Grayscale Conversion for OCR on Colored Documents
# ==============================================================
def advanced_grayscale(image):
    """
    Enhanced grayscale conversion that preserves contrast from colored text,
    backgrounds, and icons for optimal OCR accuracy.
    """
    # Convert to LAB color space to separate luminance (L) from color
    lab = cv2.cvtColor(image, cv2.COLOR_BGR2LAB)
    L, A, B = cv2.split(lab)

    # Apply CLAHE to L channel (boost local contrast)
    clahe = cv2.createCLAHE(clipLimit=2.5, tileGridSize=(8, 8))
    L_enhanced = clahe.apply(L)

    # Recombine LAB and convert back to BGR, then to grayscale
    lab_enhanced = cv2.merge([L_enhanced, A, B])
    bgr_enhanced = cv2.cvtColor(lab_enhanced, cv2.COLOR_LAB2BGR)
    gray = cv2.cvtColor(bgr_enhanced, cv2.COLOR_BGR2GRAY)

    # Local contrast boost for faint or colored text
    gray = cv2.addWeighted(gray, 1.5, cv2.GaussianBlur(gray, (0, 0), 3), -0.5, 0)
    return gray


# ==============================================================
# 🔹 Multi-Scale OCR Preprocessing
# ==============================================================
def preprocess_for_ocr_multi_scale(image_path, output_path):
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Cannot read image: {image_path}")

    gray = advanced_grayscale(img)

    # Upscale for small text clarity
    height, width = gray.shape
    upscaled = cv2.resize(gray, (width * 2, height * 2), interpolation=cv2.INTER_CUBIC)

    # Denoising while preserving edges
    denoised = cv2.fastNlMeansDenoising(upscaled, None, h=10, templateWindowSize=7, searchWindowSize=21)

    # Mild sharpening
    kernel_sharpen = np.array([[0, -1, 0],
                               [-1, 5, -1],
                               [0, -1, 0]])
    sharpened = cv2.filter2D(denoised, -1, kernel_sharpen)

    # Adaptive threshold (good for mixed font sizes)
    binary = cv2.adaptiveThreshold(sharpened, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 11, 2)

    # Morphological cleanup
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (2, 2))
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel, iterations=1)

    # Invert if necessary
    if np.mean(cleaned) < 127:
        cleaned = cv2.bitwise_not(cleaned)

    cv2.imwrite(str(output_path), cleaned)
    return cleaned


# ==============================================================
# 🔹 Large Text (Headings) Optimization
# ==============================================================
def preprocess_for_large_text(image_path, output_path):
    img = cv2.imread(str(image_path))
    gray = advanced_grayscale(img)

    # Upscale for sharper text edges
    height, width = gray.shape
    upscaled = cv2.resize(gray, (int(width * 1.5), int(height * 1.5)), interpolation=cv2.INTER_LANCZOS4)

    # Strong contrast enhancement
    clahe = cv2.createCLAHE(clipLimit=4.0, tileGridSize=(8, 8))
    enhanced = clahe.apply(upscaled)

    # Strong sharpening for bold headings
    kernel_sharpen = np.array([[-1, -1, -1],
                               [-1,  9, -1],
                               [-1, -1, -1]])
    sharpened = cv2.filter2D(enhanced, -1, kernel_sharpen)

    # Global threshold (Otsu)
    _, binary = cv2.threshold(sharpened, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

    # Light morphological cleanup
    kernel = np.ones((2, 2), np.uint8)
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    cv2.imwrite(str(output_path), cleaned)
    return cleaned


# ==============================================================
# 🔹 Hybrid Method (Balanced for Mixed Fonts)
# ==============================================================
def preprocess_hybrid(image_path, output_path):
    img = cv2.imread(str(image_path))
    if img is None:
        raise ValueError(f"Cannot read image: {image_path}")

    gray = advanced_grayscale(img)

    # Upscale moderately
    height, width = gray.shape
    upscaled = cv2.resize(gray, (width * 2, height * 2), interpolation=cv2.INTER_CUBIC)

    # Bilateral filter (keeps edges crisp)
    denoised = cv2.bilateralFilter(upscaled, 9, 75, 75)

    # CLAHE and equalization for uniform brightness
    equalized = cv2.equalizeHist(denoised)
    clahe = cv2.createCLAHE(clipLimit=3.5, tileGridSize=(8, 8))
    enhanced = clahe.apply(equalized)

    # Unsharp mask (natural sharpening)
    gaussian = cv2.GaussianBlur(enhanced, (0, 0), 1.5)
    unsharp = cv2.addWeighted(enhanced, 1.8, gaussian, -0.8, 0)

    # Adaptive threshold for mixed text sizes
    binary = cv2.adaptiveThreshold(unsharp, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
                                   cv2.THRESH_BINARY, 15, 2)

    # Morphological cleanup
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (2, 2))
    cleaned = cv2.morphologyEx(binary, cv2.MORPH_CLOSE, kernel)

    # Invert if background too dark
    if np.mean(cleaned) < 127:
        cleaned = cv2.bitwise_not(cleaned)

    cv2.imwrite(str(output_path), cleaned)
    return cleaned


# ==============================================================
# 🔹 Batch Processing for a Folder
# ==============================================================
input_folder = Path("../src/data/ground_struct/pages_images")
output_folder = Path("../src/data/ground_struct/pages_images_processed")
output_folder.mkdir(parents=True, exist_ok=True)

processed_count = 0
failed_files = []

for img_file in sorted(input_folder.glob("page_*.png")):
    output_path = output_folder / img_file.name
    try:
        preprocess_hybrid(img_file, output_path)
        processed_count += 1
        print(f"✅ Processed (hybrid): {img_file.name}")
    except Exception:
        try:
            preprocess_for_ocr_multi_scale(img_file, output_path)
            processed_count += 1
            print(f"✅ Processed (multi-scale): {img_file.name}")
        except Exception:
            try:
                preprocess_for_large_text(img_file, output_path)
                processed_count += 1
                print(f"✅ Processed (large-text): {img_file.name}")
            except Exception as e:
                failed_files.append(img_file.name)
                print(f"❌ Failed {img_file.name}: {e}")

print(f"\n📊 Summary: Processed {processed_count} images → {output_folder}")
if failed_files:
    print(f"❌ Failed ({len(failed_files)}): {failed_files}")


In [ ]:
import os

# --- Thiết lập đường dẫn ---
# Giả sử bạn đang chạy script này từ thư mục cha của 'data'
content_folder = "./data/contents"
output_file = "final_text.txt"
output_path = os.path.join("./data", output_file) # Lưu file kết quả vào thư mục 'data'

# --- 1. Lấy danh sách các tệp cần xử lý ---
# Lọc chỉ lấy các tệp .txt và sắp xếp theo số trang (ví dụ: page_1, page_2...)
all_files = os.listdir(content_folder)

# Hàm key để sắp xếp các tệp theo số trang.
# Ví dụ: "page_10.txt" sẽ đứng sau "page_9.txt"
def sort_key(filename):
    # Trích xuất số trang từ tên tệp (ví dụ: "page_1.txt" -> 1)
    try:
        return int(filename.split('_')[1].split('.')[0])
    except:
        # Đặt các tệp không theo format này ở cuối
        return float('inf') 

text_files = sorted(
    [f for f in all_files if f.endswith('.txt')],
    key=sort_key
)

# --- 2. Thực hiện nối tệp ---
print(f"Bắt đầu nối {len(text_files)} tệp từ '{content_folder}'...")

with open(output_path, "w", encoding="utf-8") as outfile:
    for filename in text_files:
        input_path = os.path.join(content_folder, filename)
        
        # Thêm tiêu đề trang (tùy chọn, để dễ phân biệt nội dung giữa các trang)
        outfile.write(f"\n")
        
        try:
            with open(input_path, "r", encoding="utf-8") as infile:
                content = infile.read()
                outfile.write(content)
                print(f"✅ Đã thêm: {filename}")
        except Exception as e:
            print(f"❌ Lỗi khi đọc tệp {filename}: {e}")

print("\n--- HOÀN TẤT ---")
print(f"Tất cả nội dung đã được lưu vào: {output_path}")

In [ ]:
from underthesea import word_tokenize
from typing import List
from utils.helper import Helpers

helpers = Helpers()

def bm25_preprocessing_func(text: str) -> List[str]:
    """
    First: MÓN GÀ, VỊT & TRỨNG, Vịt kho gừng, Thịt vịt chặt khúc, kho cùng gừng, mắm, đường cho săn, Vịt, gừng, mắm, đường, món mặn, Gừng nồng, mùi vịt kho dậy mùi, 2-3 người
    Final: ['gà', 'vịt trứng', 'vịt kho gừng', 'thịt vịt chặt khúc', 'kho gừng', 'mắm', 'đường săn', 'vịt', 'gừng', 'mắm', 'đường', 'mặn', 'gừng nồng', 'mùi vịt kho dậy mùi', '2 3 người']
    
    First: MÓN GÀ, VỊT & TRỨNG, Trứng chiên thịt, Trứng gà đánh tan, trộn thịt băm, nêm gia vị, chiên vàng, Trứng gà, thịt ba chỉ băm, món béo, mặn, Trứng chiên vàng thơm, hành lá, 2-3 người
    Final: ['gà', 'vịt trứng', 'trứng chiên thịt', 'trứng gà đánh tan', 'trộn thịt băm', 'nêm gia vị', 'chiên vàng', 'trứng gà', 'thịt ba chỉ băm', 'béo', 'mặn', 'trứng chiên vàng thơm', 'hành lá', '2 3 người']

    First: NƯỚC MÁT NHÀ LÀM, Trà đá, Trà nấu, Trà, Nước, Trà, 1 người
    Final: ['nước mát', 'trà đá', 'trà nấu', 'trà', 'nước', 'trà', '1 người']
    
    First: NƯỚC MÁT NHÀ LÀM, Coca / 7 UP, Nước ngọt có gas, Nước ngọt, Soft Drink, Gas, 1 người
    Final: ['nước mát', 'coca 7 up', 'nước ngọt gas', 'nước ngọt', 'soft drink', 'gas', '1 người']
    
    First: NƯỚC MÁT NHÀ LÀM, Nước suối, Nước suối thanh lọc, Nước suối, Nước, Nước, 1 người
    Final: ['nước mát', 'nước suối', 'nước suối thanh lọc', 'nước suối', 'nước', 'nước', '1 người']
    
    First: NƯỚC MÁT NHÀ LÀM, Bia các loại (Tiger, Heineken, Saigon), Bia các loại, Vị lúa mạch, Beer, Bia, 1 người
    Final: ['nước mát', 'bia loại tiger', 'heineken', 'saigon', 'bia loại', 'vị lúa mạch', 'beer', 'bia', '1 người']
    """
    normalized = helpers.normalize_vnese(text)
    # print("First:", normalized)
    normalized = helpers.clean_vietnamese_text(normalized)
    # print("0.1:", normalized)
    normalized = ' '.join(word_tokenize(normalized))
    # print("0.5:", normalized)
    sequences = [str(helpers.normalize_record(text=seq)) for seq in normalized.split(" , ")]
    # print("2:", sequences)
    sequences = [helpers.remove_stopwords_and_not_vi(text=seq, path_documents_vi="./data/stopwords-vietnamese.txt") for seq in sequences]
    # print("3:", sequences)
    sequences = [str(helpers.normalize_record(text=seq)) for seq in sequences]

    # sequences = [seq.replace("_", " ") for seq in sequences if seq]
    # print("Final:", sequences)
    return sequences


' '.join(bm25_preprocessing_func("NƯỚC MÁT NHÀ LÀM, Bia các loại (Tiger, Heineken, Saigon), Bia các loại, Vị lúa mạch, Beer, Bia, 1 người"))
# from underthesea import word_tokenize

# ' '.join([seq.replace(" ", "_") for seq in word_tokenize("NƯỚC MÁT NHÀ LÀM, Bia các loại (Tiger, Heineken, Saigon), Bia các loại, Vị lúa mạch, Beer, Bia, 1 người")] )

In [ ]:
from utils.helper import Helpers
from pyvi import ViTokenizer
import pandas as pd
import re
from collections import Counter
from underthesea import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt

sequences = rows

tokens = ' '.join(sequences).split()
freq = Counter(tokens)

font_path = r"C:\Windows\Fonts\arial.ttf"   # chứa tiếng Việt ổn
wc = WordCloud(width=800, height=400,
               background_color="white",
               font_path=font_path,
               collocations=False).generate_from_frequencies(freq)

plt.figure(figsize=(10, 5))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud Menu", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
from utils.helper import Helpers
from pyvi import ViTokenizer
import pandas as pd
import re
from collections import Counter
from underthesea import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt

sequences = [
    " ".join(tok if " " not in tok else tok.replace(" ", "_")
             for tok in word_tokenize(row))
    for row in rows
]
# sequences = [' '.join(bm25_preprocessing_func(seq)) for seq in sequences]

tokens = ' '.join(sequences).split()
freq = Counter(tokens)

font_path = r"C:\Windows\Fonts\arial.ttf"   # chứa tiếng Việt ổn
wc = WordCloud(width=800, height=400,
               background_color="white",
               font_path=font_path,
               collocations=False).generate_from_frequencies(freq)

plt.figure(figsize=(10, 5))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud Menu", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
print("Word total", freq.total())
print(freq.most_common(freq.total()))

In [ ]:
from utils.helper import Helpers
from pyvi import ViTokenizer
import pandas as pd
import re
from collections import Counter
from underthesea import word_tokenize
from wordcloud import WordCloud
import matplotlib.pyplot as plt

sequences = [
    " ".join(tok if " " not in tok else tok.replace(" ", "_")
             for tok in word_tokenize(row))
    for row in rows
]
sequences = [' '.join(bm25_preprocessing_func(seq)) for seq in sequences]

tokens = ' '.join(sequences).split()
freq = Counter(tokens)

font_path = r"C:\Windows\Fonts\arial.ttf"   # chứa tiếng Việt ổn
wc = WordCloud(width=800, height=400,
               background_color="white",
               font_path=font_path,
               collocations=False).generate_from_frequencies(freq)

plt.figure(figsize=(10, 5))
plt.imshow(wc, interpolation="bilinear")
plt.axis("off")
plt.title("Word Cloud Menu", fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
words, freqs = zip(*freq.most_common(freq.total()))

In [ ]:
# Vẽ biểu đồ
plt.figure(figsize=(18, 6))
plt.bar(range(len(words)), freqs, color='skyblue')
plt.title('Tần suất xuất hiện của từ')
plt.xlabel('Từ')
plt.ylabel('Tần suất')
plt.xticks(rotation=90)
plt.tight_layout()

# Vẽ biểu đồ
plt.figure(figsize=(18, 6))
plt.bar(words, freqs, color='skyblue')
plt.title('Tần suất xuất hiện của từ')
plt.xlabel('Từ')
plt.ylabel('Tần suất')
plt.xticks(rotation=90)
plt.tight_layout()

plt.show()

In [ ]:
from underthesea import pos_tag, word_tokenize
from collections import Counter
import pandas as pd

# ---------- tiền xử lý ----------
sequences = [
    " ".join(tok if " " not in tok else tok.replace(" ", "_")
             for tok in word_tokenize(row))
    for row in rows
]
sequences = [' '.join(bm25_preprocessing_func(seq)) for seq in sequences]

# ---------- chia 28 chunk ~1000 token ----------
tokens = [tok for seq in sequences for tok in seq.split()]
CHUNK = 1_000
pos_counts = Counter()

for i in range(0, len(tokens), CHUNK):
    chunk_text = ' '.join(tokens[i:i+CHUNK])
    pos_counts.update(pos for _, pos in pos_tag(chunk_text.lower()))

# ---------- tạo bảng ----------
summary = (pd.Series(pos_counts, name='token_count')
             .rename_axis('POS')
             .reset_index()
             .sort_values('token_count', ascending=False))

pos_meaning = {
    'N':'danh từ', 'V':'động từ', 'A':'tính từ', 'P':'giới từ',
    'CH':'dấu câu', 'M':'số', 'Nb':'số đếm', 'Ny':'đại từ',
    'Np': 'tên riêng',
    'R':'phó từ', 'L':'hạn định từ', 'I':'thán từ', 'E':'cảm thán',
    'C':'liên từ', 'T':'thời gian từ', 'D':'chỉ thị từ', 'Nc':'loại từ',
    'Nu': 'đại lượng đặc trưng'
}
summary['meaning'] = summary['POS'].map(pos_meaning)
summary = summary[['POS', 'meaning', 'token_count']]
print(summary.to_string(index=False))

In [ ]:
import re
from collections import Counter
from kneed import KneeLocator
import numpy as np
from utils.helper import Helpers

helpers = Helpers()


with open("./data/final_text.txt", "r", encoding="utf-8") as f:
    rows = f.read().strip().split("\n")

# # Chỉ giữ cụm có ít nhất 1 chữ cái (Việt có dấu)
rows = [line for line in rows if re.search(r"[A-Za-zÀ-ỹà-ỹ]", line)]

# sequences = [
#     " ".join(tok if " " not in tok else tok.replace(" ", "_")
#              for tok in word_tokenize(row)).strip()
#     for row in rows
# ]
sequences = [' '.join(helpers.bm25_preprocessing_func(seq)).strip() for seq in rows if seq]

counter = Counter(sequences)

# Sắp xếp giảm dần theo tần suất
sorted_lines = counter.most_common()
counts = np.array([(cnt) for line, cnt in sorted_lines])

y = np.sort(counts)[::-1]          # giảm dần
x = np.arange(len(y))
knee = KneeLocator(x, y, curve="convex", direction="decreasing")
elbow_idx = knee.elbow             # vị trí ngưỡng
threshold = y[elbow_idx]
print("Ngưỡng knee:", threshold)

# In ra
print("Top các cụm lặp lại:")
for line, cnt in sorted_lines:
    if cnt > threshold:
        print(f"{line}: {cnt}")
        
string_break = sorted_lines[0][0]

In [ ]:
import os

content_one_page = []
idx = 1

for row in rows:
    if row == string_break:
        content_one_page = content_one_page[:]
        content_one_page.append(f"---- BREAK PAGE {idx} ----")
        current_page_content = "\n".join(content_one_page)
        content_one_page = []

        os.makedirs(f'./data/contents_new', exist_ok=True)
        # Save content of current_page_content in file path: ./data/contents/page_{x}_new.txt
        with open(f'./data/contents_new/page_{idx}.txt', 'w', encoding='utf-8') as f:
            f.write(current_page_content)

        idx += 1
    else:
        content_one_page.append(row)

    # if len(current_page_content.split(" ")) > 1000:
    #     print(current_page_content)

In [ ]:
from sentence_transformers import SentenceTransformer


model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-0.6B",
    device="cuda",
    model_kwargs={"torch_dtype": "bfloat16"}
)

In [ ]:
import torch
import pandas as pd
from sentence_transformers import SentenceTransformer
from sklearn.metrics.pairwise import cosine_similarity

# 1. Load model
model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-0.6B",
    device="cuda",
    model_kwargs={"torch_dtype": torch.bfloat16}
)

# 2. Embedding toàn bộ sequences (batch_size tùy GPU)
batch_size = 64
embeddings = model.encode(
    sequences,
    batch_size=batch_size,
    show_progress_bar=True,
    convert_to_numpy=True
)

# 3. Tính ma trận tương đồng cosine
sim = cosine_similarity(embeddings)          # shape (N, N)

# 4. Ngưỡng gộp nhóm
threshold = 0.85

# 5. Gộp nhóm đơn giản bằng thuật toán Connected Component
import networkx as nx

G = nx.Graph()
for i in range(len(sequences)):
    G.add_node(i)

# Chỉ thêm cạnh có độ tương đồng >= threshold (tránh self-loop)
rows, cols = np.where(sim >= threshold)
for r, c in zip(rows, cols):
    if r != c:
        G.add_edge(r, c)

# Danh sách các thành phần liên thông
components = list(nx.connected_components(G))

# 6. Tạo DataFrame kết quả
records = []
for gid, comp in enumerate(components):
    comp = sorted(comp)                      # giữ thứ tự ban đầu
    rep_idx = comp[0]                        # chọn câu đầu làm đại diện
    for idx in comp:
        records.append({
            "group_id": gid,
            "sequence": sequences[idx],
            "representative": sequences[rep_idx]
        })

df_groups = pd.DataFrame(records)
print(df_groups.head(10))

# Xuất ra CSV nếu cần
df_groups.to_csv("similar_groups.csv", index=False, encoding="utf-8-sig")

In [ ]:
res = """**Tóm tắt nội dung báo cáo Phát triển bền vững (PTBV) của BIWASE – đầy đủ và chính xác bằng tiếng Việt:**

Từ năm 2023, BIWASE bắt đầu công bố Báo cáo Phát triển bền vững (PTBV) song hành với Báo cáo thường niên, nhằm minh bạch hóa cam kết và hành động trong lĩnh vực kinh doanh bền vững, góp phần thực hiện mục tiêu chiến lược chung của Tổng Công ty giai đoạn 2018–2025 và định hướng đến năm 2030.

Báo cáo PTBV không chỉ phản ánh kết quả thực hiện các hoạt động kinh doanh chính và các hoạt động vì môi trường – xã hội, mà còn giúp cổ đông, Hội đồng quản trị, Ban Giám đốc và các bên liên quan đánh giá một cách hệ thống về định hướng chiến lược và tiến độ triển khai kế hoạch phát triển bền vững.

- **Giai đoạn báo cáo**: Từ ngày 01/01/2024 đến hết ngày 31/12/2024. Báo cáo được lập định kỳ hàng năm.  
- **Phạm vi**: Tất cả thông tin trong báo cáo đều đến từ Công ty CP – Tổng Công ty Nước – Môi trường Bình Dương và các đơn vị thành viên.  
- **Nguồn dữ liệu**: Các số liệu được thu thập, thống kê nội bộ bởi Tổng Công ty và các đơn vị trực thuộc. Trừ khi có quy định khác, đơn vị tiền tệ sử dụng là Việt Nam Đồng (VND).  
- **Chịu trách nhiệm**: Báo cáo đã được rà soát và phê duyệt bởi Chủ tịch Hội đồng quản trị và Ban lãnh đạo vào ngày 15/03/2025.  
- **Phản hồi và liên hệ**: BIWASE luôn chào đón các ý kiến đóng góp từ các bên liên quan về hoạt động PTBV và nội dung báo cáo.  
  - **Liên hệ**: Bà Dương Anh Thư – Trưởng Ban kiểm soát  
    Email: anhthu.bwe@gmail.com  
    Điện thoại: 02743.824.245  

Báo cáo tuân thủ các quy định của Chính phủ Việt Nam, đáp ứng các tiêu chí đánh giá trong nước và từng bước tiếp cận các tiêu chuẩn quốc tế như GRI (Bộ tiêu chuẩn quốc tế về báo cáo PTBV) và SASB (tiêu chuẩn ngành dành cho lĩnh vực cấp nước và xử lý chất thải). Năm 2024 là năm thứ hai BIWASE công bố báo cáo PTBV, nên thông tin từ các năm trước còn hạn chế. Tất cả nội dung trong báo cáo đều đảm bảo nhất quán với các thông tin đã được công bố trước đó bởi Tổng Công ty.

Báo cáo được thực hiện bằng hai ngôn ngữ: tiếng Việt và tiếng Anh. Trường hợp có sự khác biệt giữa hai bản, bản tiếng Việt sẽ được xem là chuẩn."""

from underthesea import word_tokenize
from typing import List
from utils.helper import Helpers

helpers = Helpers()

def bm25_preprocessing_func(text: str) -> List[str]:

    normalized = helpers.normalize_vnese(text)
    # print("First:", normalized)
    normalized = helpers.clean_vietnamese_text(normalized)
    # print("0.1:", normalized)
    normalized = ' '.join(word_tokenize(normalized))
    # print("0.5:", normalized)
    sequences = [str(helpers.normalize_record(text=seq)) for seq in normalized.split(" , ")]
    # print("2:", sequences)
    sequences = [helpers.remove_stopwords_and_not_vi(text=seq, path_documents_vi="./data/stopwords-vietnamese.txt") for seq in sequences]
    # print("3:", sequences)
    sequences = [str(helpers.normalize_record(text=seq)) for seq in sequences]

    return sequences

res = helpers.clean_mardown_text(res)
# seq = ' '.join([seq.replace(" ", "_") for seq in word_tokenize(res)])
# ' '.join(bm25_preprocessing_func(seq)).strip()
print(res)
# seq
# helpers = Helpers()



# sequences = [' '.join(bm25_preprocessing_func(seq)).strip() for seq in sequences if seq]

# ' '.join(bm25_preprocessing_func(res))

In [ ]:
# from sentence_transformers import SentenceTransformer


# model = SentenceTransformer(
#     "Qwen/Qwen3-Embedding-0.6B",
#     device="cuda",
#     model_kwargs={"torch_dtype": "bfloat16"}
# )


summary = '''Tóm tắt nội dung báo cáo Phát triển bền vững (PTBV) của BIWASE – đầy đủ và chính xác bằng tiếng Việt: Từ năm 2023, BIWASE bắt đầu công bố Báo cáo Phát triển bền vững (PTBV) song hành với Báo cáo thường niên, nhằm minh bạch hóa cam kết và hành động trong lĩnh vực kinh doanh bền vững, góp phần thực hiện mục tiêu chiến lược chung của Tổng Công ty giai đoạn 2018–2025 và định hướng đến năm 2030
 Báo cáo PTBV không chỉ phản ánh kết quả thực hiện các hoạt động kinh doanh chính và các hoạt động vì môi trường – xã hội, mà còn giúp cổ đông, Hội đồng quản trị, Ban Giám đốc và các bên liên quan đánh giá một cách hệ thống về định hướng chiến lược và tiến độ triển khai kế hoạch phát triển bền vững
 Giai đoạn báo cáo: Từ ngày 01/01/2024 đến hết ngày 31/12/2024
 Báo cáo được lập định kỳ hàng năm
 Phạm vi: Tất cả thông tin trong báo cáo đều đến từ Công ty CP – Tổng Công ty Nước – Môi trường Bình Dương và các đơn vị thành viên
 Nguồn dữ liệu: Các số liệu được thu thập, thống kê nội bộ bởi Tổng Công ty và các đơn vị trực thuộc
 Trừ khi có quy định khác, đơn vị tiền tệ sử dụng là Việt Nam Đồng (VND)
 Chịu trách nhiệm: Báo cáo đã được rà soát và phê duyệt bởi Chủ tịch Hội đồng quản trị và Ban lãnh đạo vào ngày 15/03/2025
 Phản hồi và liên hệ: BIWASE luôn chào đón các ý kiến đóng góp từ các bên liên quan về hoạt động PTBV và nội dung báo cáo
 Liên hệ: Bà Dương Anh Thư – Trưởng Ban kiểm soát Email: anhthu.bwe@gmail.com Điện thoại: 02743.824.245 Báo cáo tuân thủ các quy định của Chính phủ Việt Nam, đáp ứng các tiêu chí đánh giá trong nước và từng bước tiếp cận các tiêu chuẩn quốc tế như GRI (Bộ tiêu chuẩn quốc tế về báo cáo PTBV) và SASB (tiêu chuẩn ngành dành cho lĩnh vực cấp nước và xử lý chất thải)
 Năm 2024 là năm thứ hai BIWASE công bố báo cáo PTBV, nên thông tin từ các năm trước còn hạn chế
 Tất cả nội dung trong báo cáo đều đảm bảo nhất quán với các thông tin đã được công bố trước đó bởi Tổng Công ty
 Báo cáo được thực hiện bằng hai ngôn ngữ: tiếng Việt và tiếng Anh
 Trường hợp có sự khác biệt giữa hai bản, bản tiếng Việt sẽ được xem là chuẩn.'''
 
 
'data\contents\page_2.txt'

In [ ]:
from sentence_transformers import SentenceTransformer
import torch

model = SentenceTransformer(
    "Qwen/Qwen3-Embedding-0.6B",
    device="cuda",
    model_kwargs={"torch_dtype": torch.bfloat16}
)

with open(r'data\contents\page_2.txt', 'r', encoding='utf-8') as f:
    page2_text = f.read()
    
import re
page2_text = re.sub(r'\s+', ' ', page2_text).strip()
    
# Embedding cho tóm tắt và nội dung page 2
summary_embedding = model.encode(summary, normalize_embeddings=True)
page2_embedding = model.encode(page2_text, normalize_embeddings=True)


from sklearn.metrics.pairwise import cosine_similarity

similarity = cosine_similarity([summary_embedding], [page2_embedding])[0][0]
print(f"Độ tương đồng cosine: {similarity:.4f}")

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch

model_name = "Qwen/Qwen2.5-1.5B-Instruct"

model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype="auto",
    device_map="auto"
)
tokenizer = AutoTokenizer.from_pretrained(model_name)

def correct_vietnamese(text: str) -> str:
    prompt = f"""Documents vietnamese:\n\n{text}"""

    messages = [
        {
            "role": "system", 
            "content": """You are a Vietnamese proofreading assistant. Your only task is to correct **spelling mistakes** in Vietnamese text while preserving the original meaning, tone, and formatting. Do not change grammar, punctuation, or style unless needed for spelling accuracy. Keep the original structure, diacritics, and formatting exactly as written. Do not translate or explain. Output only the corrected text, and never insert line breaks (\n) in your own responses.
                """.strip()},
        {"role": "user", "content": prompt}
    ]
    total_len = len(text.split())
    if total_len <= 256:
        max_new_tokens = 256
    elif total_len <= 512:
        max_new_tokens = 512
    elif total_len <= 1024:
        max_new_tokens = 1024
    elif total_len <= 2048:
        max_new_tokens = 2048
    elif total_len <= 4096:
        max_new_tokens = 4096
    else:
        max_new_tokens = 8192
    
    print(f"total {total_len} => Max new tokens:", max_new_tokens)
    
    inp = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)
    inputs = tokenizer([inp], return_tensors="pt").to(model.device)
    
    with torch.no_grad():
        out_ids = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=False,
            eos_token_id=tokenizer.eos_token_id
        )
    # lấy phần sinh sau prompt
    gen = out_ids[0][inputs.input_ids.shape[1]:]
    
    print(tokenizer.decode(gen, skip_special_tokens=True).strip())
    return tokenizer.decode(gen, skip_special_tokens=True).strip()

In [ ]:

# ---- demo ----
raw = """Báo cáo phát triển bền vững
TỔNG QUAN BÁO CÁO
Từ năm 2023, BIWASE thực hiện công bố báo cáo Phát
triển bền vững (“PTBV”) song hành cùng Báo cáo thường
niên (“BCTN”) nhằm thể hiện rõ hơn về cam kết và hành
động theo định hướng về Kinh doanh bền vững, đóng góp
vào mục tiêu chiến lược chung giai đoạn 2018-2025, định
hướng tới 2030 của Tổng Công ty.
Báo cáo PTBV không chỉ phản ánh kết quả đạt được của các hoạt động kinh doanh chính đóng
góp giá trị bền vững, các hoạt động vì môi trường - xã hội mà còn giúp Cổ đông, Hội đồng quản trị,
Ban Giám đốc và các bên liên quan khác có cơ hội nhìn lại, đánh giá một cách có hệ thống về
định hướng chiến lược, công tác triển khai theo kế hoạch kinh doanh bền vững của Công ty.
Giai đoạn báo cáo: Báo cáo PTBV năm 2024 được
sử dụng để công bố các thông tin và dữ liệu trong
giai đoạn từ 01/01/2024 đến hết 31/12/2024 và
được lập định kỳ 1 năm 1 lần.
Phạm vi báo cáo: Tất cả thông tin và dữ liệu được
công bố trong báo cáo này đến từ Công ty CP -
Tổng Công ty Nước - Môi trường Bình Dương và
các đơn vị thành viên.
Giải trình về dữ liệu: Tất cả các thông tin, số liệu trong báo cáo này đều được thu thập, thống kê
bởi nội bộ Tổng Công ty và các đơn vị trực thuộc cung cấp. Trừ khi có quy định khác, đơn vị tiền
tệ trong báo cáo này là Việt Nam Đồng (“VND”).
Chịu trách nhiệm và rà soát nội dung: Báo cáo này đã được rà soát và phê duyệt bởi Chủ tịch Hội
đồng quản trị và Ban lãnh đạo Công ty vào ngày 15/03/2025.
BIWASE luôn sẵn sàng tiếp nhận các ý kiến đóng góp liên quan đến hoạt động Phát triển bền
vững của Công ty cũng như nội dung báo cáo từ các bên liên quan.
Thông tin liên hệ: Bà Dương Anh Thư
Chức vụ: Trưởng Ban kiểm soát - Email: anhthu.bwe@gmail.com - Điện thoại: 02743.824.245
2
Thông tin được công bố trong báo cáo PTBV đảm bảo tuân theo các quy định và hướng dẫn của
Chính phủ Việt Nam, đáp ứng tối đa trong khả năng theo các tiêu chí đánh giá trong nước và từng
bước tiếp cận với các khung tiêu chuẩn quốc tế. Báo cáo tham khảo tới Bộ tiêu chuẩn quốc tế về
lập báo cáo phát triển bền vững (“GRI”) và tiêu chuẩn SASB cho ngành cấp nước và xử lý chất thải
(“SASB”). Năm 2024 là năm thứ hai BIWASE công bố báo cáo Phát triển bền vững nên thông tin
được trình bày từ Báo cáo các năm trước còn hạn chế. Mọi thông tin trong Báo cáo này đảm bảo
nhất quán với những thông tin được công bố bởi Tổng Công ty.
Báo cáo này được thực hiện bằng hai ngôn ngữ tiếng Việt và tiếng Anh, trường hợp có sự khác
biệt giữa các phiên bản ngôn ngữ thì bản tiếng Việt sẽ được lấy làm chuẩn."""

correct_vietnamese(raw)

In [ ]:

# for i, content in enumerate(pages, 1):
    # print(f"--- Trang {i} ---")
    # print(content[:150], "...")

In [ ]:
from utils.helper import Helpers
from typing import List
from underthesea import word_tokenize
import re

helpers = Helpers()

pages = [' '.join(helpers.bm25_preprocessing_func(page)) for page in pages[:30]]
pages = [page.replace("Báo cáo phát triển bền vững", " ") for page in pages if page]

# pages = [correct_vietnamese(page) for page in pages[:22]]

In [ ]:
import json
from pathlib import Path

out_dir = Path("data/summary_contents")
out_dir.mkdir(parents=True, exist_ok=True)

all_records = [
    {"id": idx, "corrected_text": text.strip()}
    for idx, text in enumerate(pages, start=1)
]

(out_dir / "all_pages.json").write_text(
    json.dumps(all_records, ensure_ascii=False, indent=2),
    encoding="utf-8"
)

print(f"✅ Đã xuất 1 file all_pages.json chứa {len(all_records)} bản ghi → {out_dir}")